# Homework 3: Problems
## Due Wednesday 16 October, by 9:30am

### PHYS 440/540, Fall 2024
https://github.com/gtrichards/PHYS_440_540/


## Problem 1
On Data Camp:

Statistical Thinking in Python (Part 2): Bootstrap confidence intervals 

This is Chapter 2.  You shouldn't need Chapter 1, but you are welcome to do it.  If this chapter interests you, then you might get something out of completing Chapters 3 and 4 as well.


## Problem 2

We're going to do some polynomial fits to data just like in the lecture.  But we didn't have time for the example in class, so please watch the Pre-Lecture video for Lecture 6 starting at 13:02 (until the end at 26:29).

Specifically, let's use the example from http://jakevdp.github.io/blog/2015/08/07/frequentism-and-bayesianism-5-model-selection/ to illustrate some ideas about goodness of fit and model selection.

Complete the cells in the starter code below to investigate how we can do both linear and polynomial fitting to two-dimensional data (y vs. x) and how to decide which of those models is "best".

Comment on which of the models has the lowest BIC.  How does that compare to our chi-square results?  What do we learn from the difference?
 

## Problem 3

Copy over all the code cells from our interactive MCMC example in Inference2 (7 cells in all from the cell that imports numpy to the histogram of the chain).  

Change the code so that the likelihood plot is drawn in purple instead of green or red when the odds ratio for the next model is not favored, but is still larger than the random value used for the "accept" threshold.  So, now you will have green plots for steps that are both better and accepted, purple for steps that aren't better but are still accepted, and red for steps that are not accepted.  Also change the code so that the step size is small enough that you are likely to take a long time to reach the most likely value.

Once you have done both of those things, then:

* Run enough steps to show 2 steps that were accepted, even though they are worse.
* Plot the chain showing it taking some time to get from the initial value to oscillating around the most likely value.
* Plot a histogram of the chain both using the full chain and after throwing away the "burn" period.


---
## Problem 2

In [ ]:
#Execute this cell to load all of the modules we'll need and define the data array.
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import optimize

In [ ]:
# generate (x,y, sigma_y) "data" 
data = np.array([[ 0.42,  0.72,  0.  ,  0.3 ,  0.15,
                   0.09,  0.19,  0.35,  0.4 ,  0.54,
                   0.42,  0.69,  0.2 ,  0.88,  0.03,
                   0.67,  0.42,  0.56,  0.14,  0.2  ],
                 [ 0.33,  0.41, -0.25,  0.01, -0.05,
                  -0.05, -0.12,  0.26,  0.29,  0.39, 
                   0.31,  0.42, -0.01,  0.58, -0.2 ,
                   0.52,  0.15,  0.32, -0.13, -0.09 ],
                 [ 0.1 ,  0.1 ,  0.1 ,  0.1 ,  0.1 ,
                   0.1 ,  0.1 ,  0.1 ,  0.1 ,  0.1 ,
                   0.1 ,  0.1 ,  0.1 ,  0.1 ,  0.1 ,
                   0.1 ,  0.1 ,  0.1 ,  0.1 ,  0.1  ]])

In [ ]:
#Functions to do a polynomial fit, compute the likelihood, and determine the best-fit parameters.

#I mostly just want you to run this cell, but see if you can understand what is going on.
#Add TWO lines of comments showing that you at least sort of understand.

def polynomial_fit(theta, x):
    """Polynomial model of degree (len(theta) - 1)"""
    # For a polynomial with order 1, this gives theta_0 + theta_1*x
    # For a polynomial with order 2, this gives theta_0 + theta_1*x + theta_2*x^2, etc.
    return sum(t * x ** n for (n, t) in enumerate(theta))

# compute the data log-likelihood given a model
def logL(theta, data, model=polynomial_fit):
    """Gaussian log-likelihood of the model at theta"""
    x, y, sigma_y = data
    y_fit = model(theta, x)
    return sum(stats.norm.logpdf(*args) for args in zip(y, y_fit, sigma_y))

# a direct optimization approach is used to get best model 
# parameters (which minimize -logL)
def best_theta(degree, model=polynomial_fit, data=data):
    theta_0 = (degree + 1) * [0]
    neg_logL = lambda theta: -logL(theta, data, model)
    return optimize.fmin_bfgs(neg_logL, theta_0, disp=False)

In [ ]:
#Execute this cell.

#Again, I mostly just want you to run this cell, but see if you can understand what is going on.
#Add TWO lines of comments showing that you at least sort of understand.

x, y, sigma_y = data
Ndata = x.size

# get best-fit parameters for linear, quadratic and cubic models
theta1 = best_theta(1, data=data)
theta2 = best_theta(2, data=data)
theta3 = best_theta(3, data=data)
# generate best fit lines on a fine grid 
xgrid = np.linspace(-0.1, 1.1, 1000)
yfit1 = polynomial_fit(theta1, xgrid)
yfit2 = polynomial_fit(theta2, xgrid)
yfit3 = polynomial_fit(theta3, xgrid)

# plot 
fig, ax = plt.subplots(figsize=(16, 8))
ax.errorbar(x, y, sigma_y, fmt='ok', ecolor='gray')
ax.plot(xgrid, yfit1, label='best linear model')
ax.plot(xgrid, yfit2, label='best quadratic model')
ax.plot(xgrid, yfit3, label='best cubic model')
ax.legend(loc='best', fontsize=14)
ax.set(xlabel='x', ylabel='y', title='data');

In [ ]:
#Complete and execute this cell to compute chi2: sum{[(y-yfit)/sigma_y]^2} 
chi21 = np.sum(((y-polynomial_fit(theta1, x))/sigma_y)**2) 
chi22 = np.sum(((y-polynomial_fit(____, x))/sigma_y)**2) 
chi23 = np.sum(((y-polynomial_fit(____, x))/sigma_y)**2) 
# normalize by the number of degrees of freedom
# the number of fitted parameters is 2, 3, 4
chi2dof1 = chi21/(Ndata - 2)
chi2dof2 = chi22/(____)
chi2dof3 = chi23/(____)

print("CHI2:")
print('   best linear model:', chi21)
print('best quadratic model:', chi22)
print('    best cubic model:', chi23)
print("CHI2 per degree of freedom:")
print('   best linear model:', chi2dof1)
print('best quadratic model:', chi2dof2)
print('    best cubic model:', chi2dof3)

In [ ]:
#Complete and execute this cell
L = [] #Array with all of the likelihood values
A = [] #Array with all of the AIC values
B = [] #Array with all of the  BICvalues
neg_LogL1 = -logL(theta1, data)+10  #+10 for plotting purposes
neg_LogL2 = -logL(theta2, data)+10
neg_LogL3 = -logL(theta3, data)+10
L = np.append(L,neg_LogL1)
L = np.append(L,neg_LogL2)
L = np.append(L,neg_LogL3)

xplot = np.arange(len(L))+1 #x values for plotting.

AIC1 = 2.*neg_LogL1 + 2.*len(theta1) + 2.*len(theta1)*(len(theta1)+1)/(len(data[0])-len(theta1)-1)
AIC2 = ____ #Complete
AIC3 = ____ #Complete
A = np.append(A,AIC1)
A = np.append(A,AIC2)
A = np.append(A,AIC3)
BIC1 = 2.*neg_LogL1 + len(theta1)*np.log(len(data[0]))
BIC2 = ____ #Complete
BIC3 = ____ #Complete
B = np.append(B,BIC1)
B = np.append(B,BIC2)
B = np.append(B,BIC3)

print("-logL:", L)
print("AIC:", A)
print("BIC:", B)


fig = plt.subplots(figsize=(16, 8))
plt.plot(xplot, L, label='Likelihood+10')
plt.plot(xplot, A, label='AIC')
plt.plot(xplot, B, label='BIC')
plt.legend()
plt.xlabel("k")
plt.ylabel("Score")
print(L)
print(B)
print(A)

---
## Problem 3

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

np.random.seed(123)

Copy the 5 cells from lecture that are between these and modify according to the instructions.

In [ ]:
ax = plt.subplot()

burn = 3000
ax.hist(posterior,bins=30,alpha=0.5,density='True',label='estimated posterior')
ax.hist(posterior[burn:],bins=30,alpha=0.5,density='True',label='estimated posterior no burn')
xplot = np.linspace(-.5, .5, 500)
post = calc_posterior_analytical(data, xplot, 0, 1)
ax.plot(xplot, post, 'g', label='analytic posterior')
_ = ax.set(xlabel='mu', ylabel='belief');
ax.legend(fontsize=10);